<a href="https://colab.research.google.com/github/carloscastillo10/predicting-academic-performance/blob/main/annex-b.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
pd.options.display.max_columns = 33
pd.set_option('mode.chained_assignment', None)

In [2]:
def load_data(path):
    data = pd.read_csv(path, low_memory=False)
    data.columns = data.columns.str.lower()
    return data

In [3]:
grades_path = 'data/notasANNONYM.csv'
grades_data = load_data(grades_path)
grades_data.head(5)

,periodo_academico,curso_codigo,curso_nombre,paralelo,paralelo_canvas,curso_canvas,aa_nota,aab01_nota,aab1_nota,aaeprb1_nota,acdapeb1_nota,acdb1_nota,apeb1_nota,cbim1_nota,vbim1_nota,bim1_nota,aab02_nota,aab2_nota,aaeprb2_nota,acdape_nota,acdapeb2_nota,acdb2_nota,apeb2_nota,cbim2_nota,vbim2_nota,bim2_nota,acu30_nota,acu65_nota,recu_nota,totalfin_nota,finalv_nota,final_nota,fake_identificacion
0,202062,DRBD_3019,COMUNICACION DE DATOS,100,202062_26189,202062_R2_DRBD_3019_1,NaN,0.64,1.95,1.31,NaN,3.33,0.51,NaN,NaN,5.79,0.63,2.01,1.38,NaN,NaN,1.35,1.55,NaN,NaN,4.91,NaN,3.37,1.93,5.4,NaN,5.3,SLXA73875862178452
1,202062,DRBD_3019,COMUNICACION DE DATOS,100,202062_26189,202062_R2_DRBD_3019_1,NaN,1.00,2.25,1.25,NaN,0.00,3.00,NaN,NaN,5.25,1.00,2.31,1.31,NaN,NaN,1.28,1.47,NaN,NaN,5.06,NaN,2.88,1.93,5.2,NaN,4.8,IMTI79985682916423
2,202062,DRBD_3019,COMUNICACION DE DATOS,100,202062_26189,202062_R2_DRBD_3019_1,NaN,0.85,2.35,1.50,NaN,1.60,2.09,NaN,NaN,6.04,0.70,2.64,1.94,NaN,NaN,2.90,1.62,NaN,NaN,7.16,NaN,4.11,2.63,6.6,NaN,6.7,YPKN05597705348880
3,202062,DRBD_3019,COMUNICACION DE DATOS,100,202062_26189,202062_R2_DRBD_3019_1,NaN,0.98,0.98,NaN,NaN,2.63,1.88,NaN,NaN,5.49,0.81,2.56,1.75,NaN,NaN,1.35,2.73,NaN,NaN,6.64,NaN,4.30,2.45,6.1,NaN,6.7,WHAX47312762710440
4,202062,DRBD_3019,COMUNICACION DE DATOS,100,202062_26189,202062_R2_DRBD_3019_1,NaN,0.92,1.80,0.88,NaN,2.43,0.00,NaN,NaN,4.23,NaN,NaN,NaN,NaN,NaN,0.00,0.00,NaN,NaN,0.00,NaN,1.22,NaN,2.1,NaN,2.1,CDKP20450612579481


# Establecer el estado final de la materia
* `>= 7` **`"APROBADA"`**
* `< 7` **`"REPROBADA"`**

In [4]:
approval_name = 'APROBADA'
disapproval_name = 'REPROBADA'

def set_final_status(final_score):    
    if final_score >= 7: 
        return approval_name
    
    if np.isnan(final_score):
        return np.NAN

    return disapproval_name
    

In [5]:
grades_data['status'] = grades_data['final_nota'].apply(set_final_status)
grades_data.to_csv('completed-grades-data.csv', index=False)
grades_data

,periodo_academico,curso_codigo,curso_nombre,paralelo,paralelo_canvas,curso_canvas,aa_nota,aab01_nota,aab1_nota,aaeprb1_nota,acdapeb1_nota,acdb1_nota,apeb1_nota,cbim1_nota,vbim1_nota,bim1_nota,...,aaeprb2_nota,acdape_nota,acdapeb2_nota,acdb2_nota,apeb2_nota,cbim2_nota,vbim2_nota,bim2_nota,acu30_nota,acu65_nota,recu_nota,totalfin_nota,finalv_nota,final_nota,fake_identificacion,status
0,202062,DRBD_3019,COMUNICACION DE DATOS,100,202062_26189,202062_R2_DRBD_3019_1,NaN,0.64,1.95,1.31,NaN,3.33,0.51,NaN,NaN,5.79,...,1.38,NaN,NaN,1.35,1.55,NaN,NaN,4.91,NaN,3.37,1.93,5.40,NaN,5.30,SLXA73875862178452,REPROBADA
1,202062,DRBD_3019,COMUNICACION DE DATOS,100,202062_26189,202062_R2_DRBD_3019_1,NaN,1.00,2.25,1.25,NaN,0.00,3.00,NaN,NaN,5.25,...,1.31,NaN,NaN,1.28,1.47,NaN,NaN,5.06,NaN,2.88,1.93,5.20,NaN,4.80,IMTI79985682916423,REPROBADA
2,202062,DRBD_3019,COMUNICACION DE DATOS,100,202062_26189,202062_R2_DRBD_3019_1,NaN,0.85,2.35,1.50,NaN,1.60,2.09,NaN,NaN,6.04,...,1.94,NaN,NaN,2.90,1.62,NaN,NaN,7.16,NaN,4.11,2.63,6.60,NaN,6.70,YPKN05597705348880,REPROBADA
3,202062,DRBD_3019,COMUNICACION DE DATOS,100,202062_26189,202062_R2_DRBD_3019_1,NaN,0.98,0.98,NaN,NaN,2.63,1.88,NaN,NaN,5.49,...,1.75,NaN,NaN,1.35,2.73,NaN,NaN,6.64,NaN,4.30,2.45,6.10,NaN,6.70,WHAX47312762710440,REPROBADA
4,202062,DRBD_3019,COMUNICACION DE DATOS,100,202062_26189,202062_R2_DRBD_3019_1,NaN,0.92,1.80,0.88,NaN,2.43,0.00,NaN,NaN,4.23,...,NaN,NaN,NaN,0.00,0.00,NaN,NaN,0.00,NaN,1.22,NaN,2.10,NaN,2.10,CDKP20450612579481,REPROBADA
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26205,202264,MATE_1108,ALGEBRA LINEAL,117,202264_48229,31195169.0,NaN,0.92,2.67,1.75,NaN,3.43,3.00,NaN,NaN,9.10,...,1.44,NaN,NaN,2.00,2.93,NaN,NaN,7.37,NaN,5.68,NaN,8.24,NaN,8.24,ZEEV69608874236753,APROBADA
26206,202264,MATE_1108,ALGEBRA LINEAL,118,202264_48461,31213968.0,NaN,0.67,2.30,1.63,NaN,2.98,2.00,NaN,NaN,7.28,...,1.25,NaN,NaN,2.70,3.00,NaN,NaN,7.95,NaN,5.34,NaN,7.62,NaN,7.62,ZIWW52070202470293,APROBADA
26207,202264,MATE_1108,ALGEBRA LINEAL,118,202264_48461,31213968.0,NaN,1.00,2.88,1.88,NaN,2.98,3.00,NaN,NaN,8.86,...,2.25,NaN,NaN,3.50,3.00,NaN,NaN,9.75,NaN,6.24,NaN,9.31,NaN,9.31,RGSD23721019241904,APROBADA
26208,202264,MATE_1108,ALGEBRA LINEAL,118,202264_48461,31213968.0,NaN,0.74,1.99,1.25,NaN,1.05,0.12,NaN,NaN,3.16,...,NaN,NaN,NaN,0.81,1.50,NaN,NaN,2.81,NaN,1.74,NaN,2.99,NaN,2.99,SXTV32701056478680,REPROBADA
